In [94]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [95]:
#pydrake imports
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import PiecewisePolynomial, InverseKinematics, Sphere, Rgba, RigidTransform, RotationMatrix, IrisInConfigurationSpace
import time
import pydrake
from pydrake.all import (SceneGraphCollisionChecker, 
                         StartMeshcat, 
                         RobotDiagramBuilder,
                         ProcessModelDirectives,
                         LoadModelDirectives,
                         MeshcatVisualizer)
from pydrake.all import GeometrySet, CollisionFilterDeclaration
from scipy.special import comb
import matplotlib.pyplot as plt
from pydrake.all import PiecewisePolynomial
from pydrake.polynomial import Polynomial as CommonPolynomial

In [96]:
# meshcat = StartMeshcat()

In [97]:
from pydrake.all import Role, MeshcatVisualizerParams
def plant_builder(usemeshcat = False):
    if usemeshcat:
        meshcat = StartMeshcat()
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    scene_graph = builder.scene_graph()
    parser = builder.parser()
    #parser.package_map().Add("cvisirisexamples", missing directory)
    if usemeshcat:
        meshcat_params = MeshcatVisualizerParams()
#         meshcat_params.role = Role.kProximity
        visualizer = MeshcatVisualizer.AddToBuilder(builder.builder(), scene_graph, meshcat,meshcat_params)
    directives_file = "7_dof_directives_newshelf.yaml"#FindResourceOrThrow() 
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    
#     collision_filter_manager = scene_graph.collision_filter_manager()
#     inspector = scene_graph.model_inspector()
#     schunk_body = plant.GetBodyByName("body")
#     iiwa_link_7 = plant.GetBodyByName("iiwa_link_7")
#     iiwa_link_6 = plant.GetBodyByName("iiwa_link_6")
#     iiwa_links = [plant.GetBodyByName(f"iiwa_link_{i}") for i in range(8)]
#     bodies = []
#     for b in iiwa_links+[schunk_body]:#[schunk_body, iiwa_link_7, iiwa_link_6]:
#         bodies += plant.GetCollisionGeometriesForBody(b)
#     geom_set = GeometrySet(bodies)
#     decl = CollisionFilterDeclaration().ExcludeWithin(geom_set)
#     collision_filter_manager.Apply(decl)

#     shelves = [plant.GetBodyByName(f"shelf{i}",
#                                   plant.GetModelInstanceByName(f"shelves{i}")) for i in range(1,3)]
#     bodies = []
#     for b in iiwa_links[:5]+shelves:
#         bodies += plant.GetCollisionGeometriesForBody(b)
#     geom_set = GeometrySet(bodies)
#     decl = CollisionFilterDeclaration().ExcludeWithin(geom_set)
#     collision_filter_manager.Apply(decl)
    
    plant.Finalize()
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    diagram.ForcedPublish(diagram_context)
    return plant, scene_graph, diagram, diagram_context, plant_context, meshcat if usemeshcat else None

plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
Ratfk = RationalForwardKinematics(plant)

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [98]:
from pydrake.geometry.optimization_dev import CspaceFreePath
q_star = np.zeros(plant.num_positions())
plane_order =1
max_degree = 1 # CHANGE THIS TO 3!!!
t0 = time.time()
cspace_free_path = CspaceFreePath(
    plant,
    scene_graph,
    q_star,
    maximum_path_degree=max_degree,
    plane_order=plane_order,
)
t1 = time.time()
print(f"Time to build collision checker {t1-t0}")

Time to build collision checker 6.662067413330078


In [100]:
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

q = np.zeros(plant.num_positions())
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)
def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(
        scene_graph, diagram_col_context
    )
query_port = scene_graph.get_query_output_port()


def check_collision_q_by_query(q):
    plant.SetPositions(plant_context, q)
    query_object = query_port.Eval(scene_graph_context)
    return 1 if query_object.HasCollisions() else 0
    

def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)


    
col_func_handle = check_collision_q_by_query#partial(eval_cons, c=collision_constraint, tol=0.01)

col_shunk_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_shunk_free =  Rgba(0.0, 0.8, 0.5, 0.5)   

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)

def showres_s(s_vis):
    showres(Ratfk.ComputeQValue(s_vis, np.zeros(7)))

def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    col = col_func_handle(qvis)
    shunk = plant.get_body(pydrake.multibody.tree.BodyIndex(9))
    tf_tot = shunk.EvalPoseInWorld(plant_context)
    tf = tf_tot.translation()
    
   # tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(7)))
    #tf = tf_tot.translation() - tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.15
#     if col:
#         meshcat.SetObject(f"/drake/visualizer/shunk",
#                                    Sphere(0.05),
#                                    col_shunk_col)
#     else:
#         meshcat.SetObject(f"/drake/visualizer/shunk",
#                                    Sphere(0.05),
#                                    col_shunk_free)
#     meshcat.SetTransform(f"/drake/visualizer/shunk",
#                                    RigidTransform(RotationMatrix(),
#                                                   tf))
    
    diagram.ForcedPublish(diagram_context)

scaler = 1 #np.array([0.8, 1., 0.8, 1, 0.8, 1, 0.8]) #do you even geometry bro?
q_min = np.array(q_low)*scaler
q_max = np.array(q_high)*scaler
q_diff = q_max-q_min

# def sample_cfree_QPoint(MAXIT=1000):
# 	it = 0
# 	while it<MAXIT:
# 		rand = np.random.rand(5)
# 		q_s = q_min + rand*q_diff
# 		col = False
# 		for _ in range(10):
# 			r  = 0.05*(np.random.rand(5)-0.5)
# 			col |= (col_func_handle(q_s+r) > 0)
# 		if not col:
# 			return q_s #Ratfk.ComputeQValue(q_s, q_star)
# 		it+=1
# 	raise ValueError("no col free point found")

idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

display(sliders[0])
for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q6', max=3.05433, min=-3.05433)

In [104]:
# from scipy.interpolate import CubicSpline
# waypoints_q =  np.array([[-2.06706, -0.3944 ,  1.43294,  0.9056 ,  0.53294, -0.8944 ,
#         0.74567],
# [-2.26706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -0.9944 ,
#         1.04567],
# [-2.96706, -0.344 ,  2.83294,  0.9056 ,  0.73294, -0.9944 ,
#         1.04567],
# [-1.06706,  0.1056 ,  2.43294,  0.9056 ,  0.73294,  0.2056 ,
#         1.04567],
# [-1.06706,  0.1056 ,  2.43294,  1.1056 ,  1.73294,  0.2056 ,
#         0.0567],
# [-1.56706,  0.1056 ,  2.43294,  1.2056 ,  1.73294,  0.8056 ,
#        -0.25433]])
# t = np.linspace(0, 1, waypoints_q.shape[0])
# coord_splines = [CubicSpline(t,waypoints_q[:,i]) for i in range(7)]
# t_refined = np.linspace(0,1,30)
# waypoints_q_refined = [[c(t) for c in coord_splines]for t in t_refined] 
# waypoints_s = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_refined])
# waypoints_q_col = waypoints_q.copy()

# # waypoints_q_col[1,:]=np.array([-2.36706, -0.3344 ,  1.43294,  0.8956 ,  0.73294, -0.9944 ,
# #         1.04567])
# waypoints_q_col[1,:]=np.array([-2.02706, 
#                                -0.3944 , 
#                                1.43294, 
#                                0.9056 ,
#                                0.73294,
#                                -0.9944 ,
#         1.04567])

# t = np.linspace(0, 1, waypoints_q_col.shape[0])
# coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
# t_refined = np.linspace(0,1,30)
# waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 

# t = np.linspace(0, 1, waypoints_q_col.shape[0])
# coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
# t_refined = np.linspace(0,1,30)
# waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 


# waypoints_s_col = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_col_refined])
# #np.array([-2.0706, -0.3244 ,  1.43294,  0.8756 ,  0.73294, -0.9944 ,
# #      1.04567])

# breaks = np.linspace(0, 1, waypoints_s.shape[0])
# samples = waypoints_s
# # traj_safe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# # polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
# traj_safe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
# polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T


# breaks = np.linspace(0, 1, waypoints_s_col.shape[0])
# samples = waypoints_s_col
# # traj_unsafe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# # polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
# traj_unsafe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
# polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T

# animate_traj(traj_safe, int(1e4), do_break = False)


In [106]:
from scipy.interpolate import CubicSpline
waypoints_q =  np.array([[-2.06706, -0.3944 ,  1.43294,  0.9056 ,  0.53294, -0.8944 ,
        0.74567],
[-2.26706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567],
[-2.96706, -0.344 ,  2.83294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567],
[-1.06706,  0.1056 ,  2.43294,  0.9056 ,  0.73294,  0.2056 ,
        1.04567],
[-1.06706,  0.1056 ,  2.43294,  1.1056 ,  1.73294,  0.2056 ,
        0.0567],
[-1.56706,  0.1056 ,  2.43294,  1.2056 ,  1.73294,  0.8056 ,
       -0.25433]])

# waypoints_q[1,:]=np.array([
#     -2.16706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -1.0944, 1.04567
# ])

t = np.linspace(0, 1, waypoints_q.shape[0])
coord_splines = [CubicSpline(t,waypoints_q[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_refined = [[c(t) for c in coord_splines]for t in t_refined] 



waypoints_s = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_refined])


waypoints_q_col = waypoints_q.copy()

waypoints_q_col[1,:]=np.array([-2.22706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567])
# waypoints_q_col[1,:]=np.array([
#     -2.06706, -0.3944 ,  1.33294,  0.9056 ,  0.73294, -0.5944 ,  0.54567
# ])
# waypoints_q_col[2,:]=np.array([
#     -2.06706, -0.3944 ,  1.33294,  0.9056 ,  0.73294, -0.5944 ,
#         0.54567
# ])

t = np.linspace(0, 1, waypoints_q_col.shape[0])
coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 

t = np.linspace(0, 1, waypoints_q_col.shape[0])
coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 


waypoints_s_col = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_col_refined])
#np.array([-2.0706, -0.3244 ,  1.43294,  0.8756 ,  0.73294, -0.9944 ,
#      1.04567])

breaks = np.linspace(0, 1, waypoints_s.shape[0])
samples = waypoints_s

traj_time = 3

# traj_safe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
traj_safe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
traj_safe.ScaleTime(traj_time)
polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T


breaks = np.linspace(0, 1, waypoints_s_col.shape[0])
samples = waypoints_s_col
# traj_unsafe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
traj_unsafe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
traj_unsafe.ScaleTime(traj_time)

polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T



In [107]:
import C_Iris_Examples.visualization_utils as vis_utils
q_star = np.zeros(plant.num_positions())
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_context, plant, plant_context,
    Ratfk, meshcat, q_star
)

num_points = int(1e4)
safe_color=Rgba(0,1,0,1)
vis_utils.visualize_s_space_trajectory(
    vis_bundle, traj_safe,
    plant.GetBodyByName("body"), #shunk,
    "/safe_traj",
    vis_utils.TrajectoryVisualizationOptions(start_size = 0.001,
                                            end_size=0.001,
                                            start_color=safe_color,
                                            end_color=safe_color,
                                            path_color=safe_color,
                                            num_points = num_points)
)

unsafe_color = Rgba(1,0,0,1)
vis_utils.visualize_s_space_trajectory(
    vis_bundle, traj_unsafe,
    plant.GetBodyByName("body"), #shunk,
    "/unsafe_traj",
    vis_utils.TrajectoryVisualizationOptions(start_size = 0.001,
                                            end_size=0.001,
                                            start_color=unsafe_color,
                                            end_color=unsafe_color,
                                            path_color=unsafe_color,
                                            num_points = num_points)
)

In [110]:
animate_traj(traj_safe, int(1e4), do_break = False)

In [109]:
animate_traj(traj_unsafe, int(1e4), do_break = False)

col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col
col


In [111]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False
cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False

## The Certifier Runs HERE

In [44]:
t0 = time.time()
stats, res = cspace_free_path.FindSeparationCertificateGivenPath(polys_safe,
                                                                 set(),
                                                                 cert_options)
t1 = time.time()
print(f"Time to certify path {t1-t0}")

Time to certify path 328.44463086128235


In [45]:
for s in stats:
    print(s.certified_safe())
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [48]:
t0 = time.time()
stats, res = cspace_free_path.FindSeparationCertificateGivenPath(polys_unsafe,
                                                                 set(),
                                                                 cert_options)
t1 = time.time()
print(f"Time to certify path {t1-t0}")

Time to certify path 330.6732964515686


In [49]:
for s in stats:
    print(s.certified_safe())
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [43]:
print(f"Total unsafe solver time = {sum([s.total_time_to_solve_progs()/20 for s in stats])}")

9212.635742750183


In [143]:
num_lin_samples = 100
t_samples = np.linspace(0, traj_safe.end_time(), num_lin_samples)
traj_lin = PiecewisePolynomial.FirstOrderHold(t_samples,
                              np.array([traj_safe.value(t) for t in t_samples]).squeeze().T)
polys_lin = np.array([traj_lin.getPolynomialMatrix(i) for i in range(len(t_samples)-1)]).squeeze().T
print(polys_lin.shape)

(7, 99)


In [ ]:
t0 = time.time()
stats, res = cspace_free_path.FindSeparationCertificateGivenPath(polys_lin,
                                                                 set(),
                                                                 cert_options)
t1 = time.time()
print(f"Time to certify path {t1-t0}")

In [121]:
for s in stats:
    print(s.certified_safe())
    

False
